In [31]:
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
df = pd.read_csv('KAG_energydata_complete.csv', sep=',')
df['date'] = pd.to_datetime(df['date'])
df['Appliances'] = df['Appliances'].astype(float)

df = df.drop(['rv1', 'rv2'], axis = 1)

cols_T = ['T1','T2','T3','T4','T5', 'T6','T7','T8','T9']
cols_H = ['RH_1','RH_2','RH_3','RH_4','RH_5', 'RH_6', 'RH_7','RH_8', 'RH_9']

In [33]:
df['HouseConsum'] = [0]*len(df)

In [34]:
def to_kW(x):
    x['Appliances'] = x['Appliances']/(1000*3.6)
    x['lights'] = x['lights']/(1000*3.6)
    x['HouseConsum'] = x['Appliances'] + x['lights']
    return x

df = df.apply(to_kW, axis=1)

In [35]:
cols = df.columns
cols = cols.drop(cols_T+cols_H)
cols = cols.drop('Appliances')
cols = cols.drop('lights')
cols

Index(['date', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'HouseConsum'],
      dtype='object')

In [36]:
df[cols].describe()

,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,HouseConsum
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,0.028194
std,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,0.028995
min,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.002778
25%,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,0.013889
50%,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,0.016667
75%,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,0.027778
max,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,0.308333


In [6]:
df[cols].to_csv('KAG_energydata_complete_2.csv')

In [54]:
keep = ['time','House overall [kW]','temperature','humidity', 'visibility', 'windSpeed','dewPoint','pressure']

In [55]:
df2 = pd.read_csv('HomeC.csv', sep=',')
cols2 = df2.columns
df2 = df2.dropna()
for i in cols2:
    if i not in keep:
        df2 = df2.drop(i, axis=1)
cols2 = df2.columns
df2.describe()

,House overall [kW],temperature,humidity,visibility,pressure,windSpeed,dewPoint
count,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000
mean,0.858962,50.741935,0.664085,9.253444,1016.301625,6.649936,38.694013
std,1.058207,19.113807,0.194389,1.611186,7.895185,3.982716,19.087939
min,0.000000,-12.640000,0.130000,0.270000,986.400000,0.000000,-27.240000
25%,0.367667,35.770000,0.510000,9.420000,1011.290000,3.660000,24.600000
50%,0.562333,50.320000,0.680000,10.000000,1016.530000,5.930000,39.030000
75%,0.970250,66.260000,0.840000,10.000000,1021.480000,8.940000,54.790000
max,14.714567,93.720000,0.980000,10.000000,1042.460000,22.910000,75.490000


In [56]:
for c in df2.columns:
    df2[c] = df2[c].astype(float)

In [57]:
def to_convert(x):
    x['time'] = 1451624400 + (x['time']-1451624400)*60
    x['windSpeed'] = x['windSpeed'] * 0.44704
    x['temperature'] = (x['temperature']-32)/1.8
    x['dewPoint'] = (x['dewPoint']-32)/1.8
    x['pressure'] = (x['pressure']/133.322368)*100
    x['visibility'] = (x['visibility']/1.609344)
    return x

df2 = df2.apply(to_convert, axis=1)

In [58]:
df2['time'] = pd.to_datetime(df2['time'], unit='s')

In [59]:
df2.describe()

,House overall [kW],temperature,humidity,visibility,pressure,windSpeed,dewPoint
count,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000
mean,0.858962,10.412186,0.664085,5.749824,762.288910,2.972787,3.718896
std,1.058207,10.618782,0.194389,1.001145,5.921876,1.780433,10.604411
min,0.000000,-24.800000,0.130000,0.167770,739.860846,0.000000,-32.911111
25%,0.367667,2.094444,0.510000,5.853317,758.529881,1.636166,-4.111111
50%,0.562333,10.177778,0.680000,6.213712,762.460205,2.650947,3.905556
75%,0.970250,19.033333,0.840000,6.213712,766.173010,3.996538,12.661111
max,14.714567,34.288889,0.980000,6.213712,781.909304,10.241686,24.161111


In [60]:
df2 = df2.drop('time', axis = 1)
cols2 = df2.columns

In [61]:
indexes = []
for i in range(0,int(len(df2[cols2[0]])/10)):
    indexes.append(i*10)

In [62]:
values = []

for p in indexes:
    values.append([df2[cols2[0]][p:p+10].mean(),df2[cols2[1]][p:p+10].mean(),df2[cols2[2]][p:p+10].mean(),df2[cols2[3]][p:p+10].mean(),df2[cols2[4]][p:p+10].mean(),df2[cols2[5]][p:p+10].mean(),df2[cols2[6]][p:p+10].mean()])


In [63]:
df2 = pd.DataFrame(values, columns = cols2)

In [64]:
df2.describe()

,House overall [kW],temperature,humidity,visibility,pressure,windSpeed,dewPoint
count,50391.000000,50391.000000,50391.000000,50391.000000,50391.000000,50391.000000,50391.000000
mean,0.858962,10.412186,0.664085,5.749824,762.288910,2.972787,3.718896
std,0.928474,10.617192,0.194233,0.998889,5.921434,1.776022,10.603960
min,0.000800,-24.800000,0.130000,0.167770,739.860846,0.000000,-32.911111
25%,0.398141,2.094444,0.510500,5.853317,758.533632,1.636390,-4.117222
50%,0.613882,10.183333,0.680000,6.213712,762.460205,2.646477,3.905556
75%,1.002376,19.033333,0.836000,6.213712,766.173010,3.992067,12.661111
max,11.224978,34.288889,0.980000,6.213712,781.909304,10.241686,24.161111


In [65]:
df2.to_csv('HomeC_3.csv')